In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn import svm
from sklearn.metrics import precision_recall_fscore_support as prfs
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.cross_validation import cross_val_score
from sklearn import linear_model
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier


C:\Users\shekh\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_csv('extrapolated_trained_data.csv')

In [3]:
df['interest_level'] = df['interest_level'].astype('category')
df['interest_level_codes'] = df['interest_level'].cat.codes

df['display_address'] = df['display_address'].astype('category')
df['display_address_codes'] = df['display_address'].cat.codes

df['latitude'] = df['latitude'].astype('category')
df['latitude_codes'] = df['latitude'].cat.codes

df['longitude'] = df['longitude'].astype('category')
df['longitude_codes'] = df['longitude'].cat.codes

In [4]:
featuresDF = df.drop('interest_level', axis=1)
featuresDF = featuresDF.drop('building_id', axis=1)
featuresDF = featuresDF.drop('created', axis=1)
featuresDF = featuresDF.drop('description', axis=1)
featuresDF = featuresDF.drop('display_address', axis=1)
featuresDF = featuresDF.drop('latitude', axis=1)
featuresDF = featuresDF.drop('longitude', axis=1)
featuresDF = featuresDF.drop('manager_id', axis=1)
featuresDF = featuresDF.drop('photos', axis=1)
featuresDF = featuresDF.drop('street_address', axis=1)
featuresDF = featuresDF.drop('listing_id', axis=1)
featuresDF = featuresDF.drop('Unnamed: 328', axis=1)

In [5]:
y_label = featuresDF['interest_level_codes']
y_label.value_counts()

1    32216
2     9952
0     3174
Name: interest_level_codes, dtype: int64

In [6]:
target_names = ['0', '1', '2'];
X_train, X_test, y_train, y_test = train_test_split(featuresDF,y_label, test_size=0.25,random_state=15,stratify=y_label)
print len(X_train)
print len(X_test)
print len(y_train)
print len(y_test)

34006
11336
34006
11336


In [7]:
def generate_rf(X_train, y_train, X_test, y_test):
    rf = RandomForestClassifier(n_estimators=50, min_samples_leaf=10)
    rf.fit(X_train, y_train)
    print "rf score ", rf.score(X_test, y_test)
    return rf

def combine_rfs(rf_a, rf_b):
    rf_a.estimators_ += rf_b.estimators_
    rf_a.n_estimators = len(rf_a.estimators_)
    return rf_a

In [8]:
rfs = [generate_rf(X_train, y_train, X_test, y_test) for i in xrange(10)]
rf_combined = reduce(combine_rfs, rfs)

print "rf combined score", rf_combined.score(X_test, y_test)

rf score  0.924488355681
rf score  0.922194777699
rf score  0.929693013409
rf score  0.914608327452
rf score  0.926517290049
rf score  0.923959068454
rf score  0.928281580805
rf score  0.932251235004
rf score  0.9268701482
rf score  0.928899082569
rf combined score 0.92845800988


In [9]:
predicted_labels = rf_combined.predict(X_test)

In [10]:
actual_class = y_test.tolist()
predicted_class = predicted_labels.tolist()

In [11]:
target_names = ['0', '1', '2']

In [12]:
print(classification_report(actual_class, predicted_class, target_names=target_names))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00       794
          1       0.91      1.00      0.95      8054
          2       1.00      0.99      1.00      2488

avg / total       0.86      0.93      0.90     11336



C:\Users\shekh\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
#predicted_prob_labels = multinomialNBClassifier.predict_proba(X_test);

In [14]:
score = rf_combined.score(X_test,actual_class);
print score;

0.92845800988


In [15]:
cvScores = cross_val_score(rf_combined, featuresDF, y_label, cv=10, scoring='f1_micro');
print cvScores;

[ 0.92791005  0.92768959  0.92833517  0.92546858  0.92655492  0.92655492
  0.92653872  0.92808295  0.92764174  0.92874476]


In [16]:
#print len(predicted_prob_labels)
#prob_labels_list = predicted_prob_labels.tolist()
#print prob_labels_list